# Pipeline de Preprocesamiento
 
En este notebook implementaremos el pipeline de preprocesamiento para los datos de bancarrota, incluyendo:
- Normalización de variables
- Manejo de outliers
- Balanceo de clases
- Guardado de datos procesados

In [2]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import os

## 1. Carga de Datos

In [3]:
print("Cargando datos...")
train_data = pd.read_csv('../archivos/train_data.csv')
test_data = pd.read_csv('../archivos/test_data.csv')

# Mostrar dimensiones iniciales
print(f"\nDimensiones del conjunto de entrenamiento: {train_data.shape}")
print(f"Dimensiones del conjunto de prueba: {test_data.shape}")

Cargando datos...

Dimensiones del conjunto de entrenamiento: (2050, 65)
Dimensiones del conjunto de prueba: (500, 64)


## 2. Separación de Características y Objetivo

In [4]:
# Separar características y objetivo
X_train = train_data.drop(['ID', 'Bankruptcy'], axis=1)
y_train = train_data['Bankruptcy']
X_test = test_data.drop('ID', axis=1)

print("\nDimensiones después de la separación:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")


Dimensiones después de la separación:
X_train: (2050, 63)
y_train: (2050,)
X_test: (500, 63)


## 3. Identificación de Variables Numéricas


In [5]:
# Identificar variables numéricas
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNúmero de variables numéricas: {len(numeric_features)}")
print("\nVariables numéricas:")
print(numeric_features)


Número de variables numéricas: 63

Variables numéricas:
['Realized.Sales.Gross.Profit.Growth.Rate', 'Operating.Expense.Rate', 'Operating.Profit.Per.Share..Yuan...', 'Gross.Profit.to.Sales', 'Tax.rate..A.', 'Current.Liability.to.Liability', 'Continuous.Net.Profit.Growth.Rate', 'Long.term.Liability.to.Current.Assets', 'Total.assets.to.GNP.price', 'Current.Liabilities.Equity', 'CFO.to.Assets', 'Current.Liability.to.Current.Assets', 'Persistent.EPS.in.the.Last.Four.Seasons', 'Cash.Total.Assets', 'Inventory.Working.Capital', 'Net.Value.Per.Share..B.', 'Current.Assets.Total.Assets', 'Net.Worth.Turnover.Rate..times.', 'Interest.Coverage.Ratio..Interest.expense.to.EBIT.', 'Equity.to.Liability', 'Operating.profit.Paid.in.capital', 'No.credit.Interval', 'Interest.bearing.debt.interest.rate', 'Net.Income.to.Stockholder.s.Equity', 'Quick.Ratio', 'Cash.Flow.to.Sales', 'Equity.to.Long.term.Liability', 'Working.Capital.Equity', 'After.tax.net.Interest.Rate', 'Current.Liability.to.Assets', 'Net.Value

## 4. Creación del Pipeline de Preprocesamiento


In [6]:
# Crear pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numeric_features)
    ],
    remainder='passthrough'
)

# Crear pipeline completo con SMOTE
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42))
])

## 5. Aplicación del Pipeline


In [7]:
# Aplicar pipeline
print("\nAplicando pipeline de preprocesamiento...")
X_train_processed, y_train_processed = pipeline.fit_resample(X_train, y_train)

# Mostrar dimensiones después del procesamiento
print(f"\nDimensiones después del procesamiento:")
print(f"X_train_processed: {X_train_processed.shape}")
print(f"y_train_processed: {y_train_processed.shape}")

# Procesar conjunto de prueba
X_test_processed = pipeline.named_steps['preprocessor'].transform(X_test)
print(f"X_test_processed: {X_test_processed.shape}")


Aplicando pipeline de preprocesamiento...

Dimensiones después del procesamiento:
X_train_processed: (3976, 63)
y_train_processed: (3976,)
X_test_processed: (500, 63)


## 6. Guardado de Datos Procesados


In [8]:
# Guardar datos procesados
print("\nGuardando datos procesados...")
os.makedirs('archivos', exist_ok=True)

np.save('archivos/X_train_processed.npy', X_train_processed)
np.save('archivos/y_train_processed.npy', y_train_processed)
np.save('archivos/X_test_processed.npy', X_test_processed)

print("\nDatos procesados guardados exitosamente:")
print("- X_train_processed.npy")
print("- y_train_processed.npy")
print("- X_test_processed.npy")


Guardando datos procesados...

Datos procesados guardados exitosamente:
- X_train_processed.npy
- y_train_processed.npy
- X_test_processed.npy


## 7. Verificación de Balance de Clases


In [9]:
# Verificar balance de clases
print("\nDistribución de clases después del balanceo:")
print(pd.Series(y_train_processed).value_counts(normalize=True))


Distribución de clases después del balanceo:
Bankruptcy
0    0.5
1    0.5
Name: proportion, dtype: float64


## 8. Resumen del Procesamiento


In [10]:
print("\nResumen del procesamiento:")
print(f"1. Datos originales: {X_train.shape[0]} muestras")
print(f"2. Datos procesados: {X_train_processed.shape[0]} muestras")
print(f"3. Variables numéricas procesadas: {len(numeric_features)}")
print(f"4. Balance de clases logrado: Sí")
print(f"5. Datos de prueba procesados: {X_test_processed.shape[0]} muestras")


Resumen del procesamiento:
1. Datos originales: 2050 muestras
2. Datos procesados: 3976 muestras
3. Variables numéricas procesadas: 63
4. Balance de clases logrado: Sí
5. Datos de prueba procesados: 500 muestras
